# Benchmarking the GRAPES Deep Learning EEW Model on a Suite of California Earthquakes

Earthquake Early Warning (EEW) systems are designed to give people a heads-up before strong shaking starts. Traditional systems predict shaking by figuring out the size and location of an earthquake, but this can take a few seconds, potentially delaying the warning. The GRAPES EEW algorithm uses a new approach, powered by deep learning, to make faster and more accurate predictions. It looks at data from seismic sensors and connects them in a network, learning how shaking travels between stations. GRAPES analyzes patterns in the data every 4 seconds and updates predictions in real time.

Originally trained on Japanese earthquake data, we're now testing GRAPES on California earthquakes to see how well it works here. In one test with the 2019 M7.1 Ridgecrest earthquake , the system warned all stations that needed alerts, with warning times ranging from immediate at the epicenter to over 30 seconds for stations further away. This shows the potential for GRAPES to help save lives by giving people more time to take cover. We’re continuing to evaluate it on 40 additional California earthquakes to measure its accuracy and warning times.

The faster and more reliable these systems get, the more time people have to protect themselves before the shaking starts.

## 2019 M7.1 Ridgecrest Earthquake Test

![My Image](../../../Desktop/Ridgecrest.png)

### Modules & Packages

In [1]:
# I had to set SSL paths to install GRAPES.jl from Gitlab - this is an issue with Python / government firewalls
import os
os.environ["JULIA_SSL_CA_ROOTS_PATH"] = ""

# this imports julia as a python module called jl 
from juliacall import Main as jl

# pkg is the Julia package installer 
from juliacall import Pkg as Pkg

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
# load required julia packages into jl path 
jl.seval("using GRAPES") 
jl.seval("using GraphNeuralNetworks")

In [3]:
import obspy
from obspy import read
import numpy as np
from collections import Counter
from obspy import Stream
from obspy.geodetics import locations2degrees
from datetime import datetime
import subprocess
from scipy.signal import resample
import glob
from obspy.core.inventory import Inventory, Network, Station
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
from obspy import UTCDateTime
from obspy import read_inventory

### Processing and Filtering Seismic Event Data

The below code chunk downloads waveform data for specific seismic events from an AWS S3 bucket, organizes the data into directories based on the event date, and saves it locally in the specified directory.

In [4]:
#Downloading event wavefrom data

# List of formatted dates and event IDs
formatted_dates = [
    "2019/2019_187/"
]

event_ids = [
    "38457511",
]

# Working directories
data_dir = "../data/scedc/event_waveforms"

# Download each event's data
for date_dir, event_id in zip(formatted_dates, event_ids):
    day_dir = os.path.join(data_dir, date_dir)
    os.makedirs(day_dir, exist_ok=True)
    ms_filename = f"{event_id}.ms"
    print(f"Downloading event {ms_filename} {datetime.now()}")

    # Get file path
    s3_path = os.path.join("s3://scedc-pds/event_waveforms/", date_dir, ms_filename)
    local_path = os.path.join(day_dir, ms_filename)

    aws_str = f'aws s3 cp {s3_path} {local_path} --no-sign-request'
    subprocess.call(aws_str, shell=True)

download: s3://scedc-pds/event_waveforms/2019/2019_187/38457511.ms to ../data/scedc/event_waveforms/2019/2019_187/38457511.ms


The below code chunk focuses on preprocessing seismic waveform data for multiple events, ensuring data quality and consistency. It iterates through directories containing waveform files, filters by network and channel, and retains only stations with complete component data (East, North, Vertical). In this preliminary test, the code successfully processed around 900 traces, demonstrating its ability to clean and standardize data effectively. The process includes synchronizing traces by aligning start times, resampling data to a uniform 100 Hz sampling rate, and converting units from m/s² to cm/s². Stations with incomplete or duplicate data are excluded, and the final cleaned dataset for each event is stored in a dictionary for further analysis. In the overall test, the code scaled to process over 10,000 traces, highlighting its robustness and efficiency in managing large volumes of seismic data, ensuring that the data is thoroughly prepared for subsequent modeling or analysis.

In [5]:
#Preprocessing of the seismic data for each event

# Base directory containing the waveform data
base_directory = "../data/scedc/event_waveforms/"

# Dictionary to store streams for each event
event_streams = {}

# Iterate through each year folder
for year in os.listdir(base_directory):
    year_directory = os.path.join(base_directory, year)
   
    if os.path.isdir(year_directory):
        # Iterate through each event folder
        for event in os.listdir(year_directory):
            event_directory = os.path.join(year_directory, event)
           
            if os.path.isdir(event_directory):
                # Initialize an empty Stream object for the event
                st = obspy.Stream()
               
                # Iterate through all files in the event directory
                for filename in os.listdir(event_directory):
                    if filename.endswith(".ms"):
                        file_path = os.path.join(event_directory, filename)
                        try:
                            st += read(file_path)
                        except Exception as e:
                            print(e)

                st = st.select(network="CI")
                st.merge()
                st = st.select(channel="HN*")
                st = st.select(location="")

                stations_to_remove = {"PDM"}

                st.traces = [trace for trace in st if trace.stats.station not in stations_to_remove]

                station_components = {}
                # Count the components for each station
                for trace in st:
                    station = trace.stats.station
                    component = trace.stats.channel[-1]  # Last character indicates the component (E, N, Z)
                    if station not in station_components:
                        station_components[station] = set()
                    station_components[station].add(component)
               
                # Keep only stations with all three components (E, N, Z)
                stations_with_all_components = {station for station, components in station_components.items() if {'E', 'N', 'Z'} <= components}
               
                # Filter the stream manually for stations with all components and non-zero data
                filtered_traces = [trace for trace in st
                                   if trace.stats.station in stations_with_all_components and
                                      not all(x == 0 for x in trace.data)]
               
                # Sort traces alphabetically by station code
                sorted_traces = sorted(filtered_traces, key=lambda x: x.stats.station)
               
                # Create a new Stream from the filtered traces
                st = st.__class__(sorted_traces)

                st = st.split()

                # Extract station names from each Trace
                station_names = [trace.stats.station for trace in st]
               
                # Count the frequency of each station
                station_counts = Counter(station_names)
               
                # Filter stations that appear more than 3 times
                frequent_stations = {station: count for station, count in station_counts.items() if count > 3}
               
                # Filter out traces from the frequent stations
                filtered_traces = [trace for trace in st if trace.stats.station not in frequent_stations]
               
                st = Stream(traces=filtered_traces)

                # Create a new Stream for adjusted traces
                adjusted_traces = []
               
                # Iterate through the Stream in batches of 3 traces
                for i in range(0, len(st), 3):
                    # Get the next 3 traces (assuming they are grouped by station)
                    traces = st[i:i+3]
                   
                    if len(traces) == 3:
                        # Find the maximum start time among these 3 traces
                        max_start_time = max(trace.stats.starttime for trace in traces)
                       
                        # Trim all traces to start at the maximum start time
                        for trace in traces:
                            trace.trim(starttime=max_start_time)
                            adjusted_traces.append(trace)
               
                # Create a new Stream with adjusted traces
                st = Stream(traces=adjusted_traces)

                #Checking sampling rate
                for tr in st:
                    if tr.stats.sampling_rate != 100:
                        num_samples = int(tr.stats.npts * (100 / tr.stats.sampling_rate))
                        tr.data = resample(tr.data, num_samples)
                        tr.stats.sampling_rate = 100

                #Converting from m/s^2 to cm/s^2
                for tr in st:
                    tr.data *= 100
                   
                # Store the raw stream in the event_streams dictionary
                event_streams[event] = st

![My Image](../../../Desktop/Form.png)

Example of a single waveform from the east channel of station ADO. This waveform captures the motion induced by the earthquake.

In [6]:
# List of ObsPy UTCDateTime objects representing the specific dates and times of seismic events.
dates = [
    obspy.UTCDateTime(2019, 7, 6, 3, 19, 53)
]

This code chunk reads multiple StationXML files, which contain seismic station metadata, and combines them into a single Inventory object, specifically to gather station locations and remove sensitivities from the waveform data. By iterating through the files and appending the station data to the new network, the code ensures all relevant station locations are centralized. The resulting Inventory object provides a streamlined resource for working with station metadata, simplifying further seismic analysis by focusing on station locations while excluding sensitivities.

In [7]:
# Path to the directory containing the StationXML files
stationxml_directory_path = "../data/scedc/FDSNstationXML/CI/*.xml"

# Read all StationXML files in the directory into Inventory objects
stationxml_files = glob.glob(stationxml_directory_path)

#Initialize an empty Network object
inventory = Inventory(networks=[], source="Combined StationXML")

# Initialize a single Network object
network_code = "CI"
network = Network(code=network_code, stations=[])

# Loop through the StationXML files and add them to the Network object
for stationxml_file in stationxml_files:
    # Read the StationXML file
    inv = read_inventory(stationxml_file)
   
    # Loop through the stations in the Inventory and add them to the new network
    for net in inv:
        if net.code == network_code:
            network.stations.extend(net.stations)

# Add the network to the inventory
inventory.networks.append(network)

In [8]:
#  Fruther preprocessing of the seismic data for each event:
# 1. Remove any constant trend from the data.
# 2. Apply a cosine taper to the first and last 5% of the data.
# 3. Apply a high-pass filter with a cutoff frequency of 0.25 Hz to remove low-frequency noise.
# 4. Correct the data using the station's instrument response based on the provided inventory.
for event_key, st in event_streams.items():
    st.detrend("constant")
    st.taper(max_percentage=0.05, type='cosine')
    st.filter("highpass", freq=0.25)
    st.remove_sensitivity(inventory=inventory)

### Functions

The following functions organize and prepare the data for processing by the GRAPES model, which relies on three critical inputs. First, the model requires a graph of station nodes, where nodes represent individual seismic stations linked by their geographic proximity and neighboring connections. Second, it needs acceleration waveforms that are segmented into 4-second windows and formatted into a 4-dimensional tensor to capture temporal dynamics effectively. Lastly, the model incorporates current maximum acceleration (PGA) values for each station, providing real-time insights into the intensity of seismic activity. These functions ensure that each input is accurately processed and structured to optimize the model's performance and predictive capabilities.

In [9]:
def window_arrays(origin):
    """
    Generate arrays of start and end times for 4-second windows.

    This function creates two arrays: one for the start times and one for the end times of 4-second windows, 
    starting from the given origin time. The windows are generated sequentially with a 1-second step 
    from the origin up to 117 seconds later.

    Args:
        origin (obspy.UTCDateTime): The starting time for the windows.

    Returns:
        tuple: A tuple containing two NumPy arrays:
            - start_of_window (np.ndarray): Array of start times for each window.
            - end_of_window (np.ndarray): Array of end times for each window.

    Notes:
        The length of both arrays will be 118, where each window is 4 seconds long, 
        and the start times are spaced 1 second apart.
    """
    start_of_window = []
    end_of_window = []

    i = 1
    start = origin
    while start != origin + 117:
        start_of_window.append(start)
        end_of_window.append(start+4)
        start += 1

    start_of_window = np.array(start_of_window, dtype=obspy.UTCDateTime)
    end_of_window = np.array(end_of_window, dtype=obspy.UTCDateTime)

    return start_of_window, end_of_window

In [10]:
def window(t1, t2, st):
    """
    Extract a segment from a seismic Stream based on specified time window.

    This function creates a new `Stream` object by copying the input stream and trimming it to the specified 
    start and end times. The resulting stream will only contain data within the given time window.

    Args:
        t1 (obspy.UTCDateTime): The start time of the desired window.
        t2 (obspy.UTCDateTime): The end time of the desired window.
        st (obspy.Stream): The input `Stream` object to be trimmed.

    Returns:
        obspy.Stream: A new `Stream` object containing data between `t1` and `t2`.

    Notes:
        The original `Stream` is not modified; a new `Stream` is returned.
    """
    stream = st.copy().trim(starttime=t1, endtime=t2)
    
    return stream

In [11]:
def sample_adjust_trace(tr, target):
    """
    Adjust the number of samples in a trace to match a target size.

    This function modifies the input trace to ensure it has exactly the specified number of samples. 
    If the trace has more samples than the target, it will be trimmed. If it has fewer samples, it 
    will be padded with zeros.

    Args:
        tr (obspy.Trace): The input trace to be adjusted.
        target (int): The target number of samples for the trace.

    Returns:
        obspy.Trace: The modified trace with the number of samples adjusted to the target size.

    Notes:
        - The trace's `npts` attribute is updated to reflect the new number of samples.
        - The trace's data is either trimmed or padded with zeros as necessary.
    """
    if tr.stats.npts > target:
        # Trim the trace to the target number of samples
        tr.data = tr.data[:target]
    elif tr.stats.npts < target:
        # Pad the trace with zeros to the target number of samples
        padding = target - tr.stats.npts
        tr.data = np.pad(tr.data, (0, padding), 'constant', constant_values=0)
    # Update the number of samples
    tr.stats.npts = target
    return tr

In [12]:
def sample_adjust_stream(target, st):
    """
    Adjust the number of samples in each trace of a Stream to match a target size.

    This function iterates through each trace in the input Stream and adjusts its number of samples 
    to the specified target size. Traces with the target number of samples are kept as they are, while 
    those with a different number of samples are adjusted using the `sample_adjust_trace` function. 
    A new `Stream` object is created with the adjusted traces.

    Args:
        target (int): The target number of samples for each trace.
        st (obspy.Stream): The input Stream containing the traces to be adjusted.

    Returns:
        obspy.Stream: A new Stream object with traces adjusted to the target number of samples.

    Notes:
        - The function uses `sample_adjust_trace` to handle the adjustment of individual traces.
        - The original Stream is not modified; a new Stream with adjusted traces is returned.
    """
    # Check and adjust each trace
    adjusted_traces = []
    for tr in st:
        if tr.stats.npts == target:
            adjusted_traces.append(tr)
        else:
            # Adjust the trace
            adjusted_traces.append(sample_adjust_trace(tr, target))

    #create a new Stream with the adjusted traces
    st = Stream(traces=adjusted_traces)
    return st

In [13]:
def num_unique_stations(st):
    """
    Count the number of unique stations in a Stream.

    This function extracts station names from each trace in the input Stream and calculates the number of 
    unique stations by counting distinct station names.

    Args:
        st (obspy.Stream): The input Stream containing traces from various stations.

    Returns:
        int: The number of unique stations in the Stream.

    Notes:
        - The function assumes that each trace in the Stream has a `station` attribute in its `stats`.
    """
    # Extract station names
    station_names = [tr.stats.station for tr in st]

    # Count unique stations
    unique_stations = set(station_names)
    number_of_unique_stations = len(unique_stations)

    return number_of_unique_stations

In [14]:
def distance_and_location(st, inventory, origin_lat, origin_long):
    """
    Calculate distances from the epicenter and station locations.

    This function computes the distance from a given epicenter to each seismic station, using only every 
    third trace (index 2) in the input Stream since each station contains 3 traces. It also collects the latitude and longitude of each station.

    Args:
        st (obspy.Stream): The input Stream containing traces from various stations.
        inventory (obspy.Inventory): The inventory of seismic stations, including their coordinates.
        origin_lat (float): The latitude of the epicenter.
        origin_long (float): The longitude of the epicenter.

    Returns:
        tuple: A tuple containing three lists:
            - dist_from_epicenter_single (list): List of distances from the epicenter for each station.
            - dist_from_epicenter_full (list): List of distances from the epicenter for each station's trace.
            - station_location (list): List of tuples representing the latitude and longitude of each station.

    Notes:
        - The distance is calculated using the `locations2degrees` function.
        - Only every third trace (based on zero-index count) is used for distance calculations.
        - The `inventory` object is assumed to be organized by network and station.
    """
    dist_from_epicenter_single = []
    dist_from_epicenter_full = []
    station_location = []
    prev_station = ""
    trace_count = 0
    
    for network in inventory:
        for station in network:
            for trace in st.select(station=station.code):
                if trace_count % 3 == 2:  # Every third trace (index 2)
                    x = locations2degrees(origin_lat, origin_long, station.latitude, station.longitude)
                    y = (station.latitude, station.longitude)
                    
                    dist_from_epicenter_single.append(x)
                    dist_from_epicenter_full.append(x)
                    dist_from_epicenter_full.append(x)
                    dist_from_epicenter_full.append(x)
                    station_location.append(y)
                trace_count += 1
                
    return dist_from_epicenter_single, dist_from_epicenter_full, station_location

In [15]:
def sorter(dist_from_epicenter_single, dist_from_epicenter_full, station_location, st):
    """
    Sort traces and associated data by distance from the epicenter.

    This function sorts a Stream of seismic traces and associated distance arrays based on distances from the epicenter.
    The traces are ordered according to their distances, and the corresponding distance and location arrays are 
    re-ordered to match this sorting.

    Args:
        dist_from_epicenter_single (list): List of distances from the epicenter for each station.
        dist_from_epicenter_full (list): List of distances from the epicenter for each station's trace.
        station_location (list of tuples): List of tuples representing the latitude and longitude of each station.
        st (obspy.Stream): The input Stream containing seismic traces that need to be sorted.

    Returns:
        tuple: A tuple containing:
            - ordered_stream (obspy.Stream): The Stream sorted by distance from the epicenter.
            - dist_from_epicenter_single_array (np.ndarray): Array of distances distances from the epicenter for each station, sorted.
            - dist_from_epicenter_full_array (np.ndarray): Array of distances from the epicenter for each station's trace, sorted.
            - station_location_array (np.ndarray): Array of tuples representing the latitude and longitude of each station, sorted.

    Notes:
        - The sorting is performed based on the distances provided in `dist_from_epicenter_full` and 'dist_from_epicenter_single'.
        - The distances and locations are converted to NumPy arrays for efficient indexing and sorting.
    """
    dist_idx = np.argsort(dist_from_epicenter_full)
    dist_idx_2 = np.argsort(dist_from_epicenter_single)

    #Sorting stream by distance
    ordered_stream = obspy.Stream([st[ii] for ii in dist_idx])

    #Sorting distance array
    dist_from_epicenter_single_array = np.array(dist_from_epicenter_single, dtype=float)
    dist_from_epicenter_single_array = dist_from_epicenter_single_array[dist_idx_2] 

    #Sorting distance total array
    dist_from_epicenter_full_array = np.array(dist_from_epicenter_full, dtype=float)
    dist_from_epicenter_full_array = dist_from_epicenter_full_array[dist_idx] 

    #Sorting station location array
    station_location_array = np.array(station_location, dtype=float)
    station_location_array = station_location_array[dist_idx_2]

    return ordered_stream, dist_from_epicenter_single_array, dist_from_epicenter_full_array, station_location_array

In [16]:
def distance_matrix(station_location_array):
    """
    Compute a matrix of distances between pairs of stations.

    This function calculates the distance between each pair of stations using their latitude and longitude 
    coordinates and returns a distance matrix. The distance is computed using the `locations2degrees` function.

    Args:
        station_location_array (list of tuples): A list where each tuple contains the latitude and longitude 
            of a station in the form (latitude, longitude).

    Returns:
        np.ndarray: A 2D NumPy array where each element [i, j] represents the distance between the i-th and j-th
            stations, calculated using the `locations2degrees` function.

    Notes:
        - The distance matrix will be square, with dimensions equal to the number of stations.
        - The function assumes that `station_location_array` contains valid latitude and longitude pairs.
    """
    dist_matrix = np.zeros((len(station_location_array), len(station_location_array)))
    for i in range(len(station_location_array)):
        for j in range(len(station_location_array)):
            dist_matrix[i][j] = locations2degrees(station_location_array[i][0], station_location_array[i][1], station_location_array[j][0], station_location_array[j][1])
    return dist_matrix

In [17]:
def adjacency_matrix(dist_matrix, station_location_array):
    """
    Generate an adjacency matrix based on the k-nearest neighbors of each station.

    This function creates an adjacency matrix where each entry indicates whether two stations are 
    connected based on their k-nearest neighbors.

    Args:
        dist_matrix (np.ndarray): A 2D NumPy array representing the distance matrix, where each element [i, j]
            denotes the distance between the i-th and j-th stations.
        station_location_array (list of tuples): A list of tuples containing the latitude and longitude
            of each station.

    Returns:
        np.ndarray: A 2D NumPy array (adjacency matrix) where each element [i, j] is 1 if the i-th and j-th 
            stations are among each other's 10 nearest neighbors, and 0 otherwise.

    Notes:
        - The adjacency matrix is symmetric, meaning if station i is connected to station j, then station j 
            is also connected to station i.
        - The function assumes that `dist_matrix` is a square matrix with the number of stations as its dimensions.
    """
    #Creating k = 10 adjacency matrix
    adj_matrix = np.zeros_like(dist_matrix)

    for i in range(len(station_location_array)):
        # Get the indices of the k nearest neighbors, excluding the point itself
        nearest_neighbors = np.argsort(dist_matrix[i])[1:11]
        
        # Set the corresponding entries in the adjacency matrix to 1
        adj_matrix[i, nearest_neighbors] = 1
        adj_matrix[nearest_neighbors, i] = 1

    return adj_matrix

![My Image](../../../Desktop/Graph.png)

The GRAPES model utilizes a Graph Neural Network (GNN) to predict earthquake shaking by leveraging both station-level and network-level information. At each seismic station (represented as nodes in the graph), real-time waveform data is processed to extract key features, such as phase and amplitude, forming feature vectors that represent the waveforms. These feature vectors are then shared with neighboring stations using the GNN, where each station communicates with its nearby stations (within K hops, such as K=1 or K=2). For the test, K=10 neighbors were used, and this was implemented by creating an adjacency matrix to define the relationships between stations. The GNN aggregates and updates the features from its neighbors, refining the information available at each node. Finally, the model predicts Peak Ground Acceleration (PGA) at each station using the refined features, allowing the system to estimate shaking intensity in real time as the earthquake rupture evolves. This combination of local waveform data and network-wide feature sharing helps improve the prediction of ground shaking across the region.

In [18]:
def tensor_maker(num_unique_stations, st):
    """
    Create a 4D tensor from seismic trace data for multiple stations.

    This function generates a tensor of shape `(400, 3, 1, num_unique_stations)`, where:
    - The first dimension (400) represents the number of samples in each trace (assuming each trace has 400 data points).
    - The second dimension (3) corresponds to the three seismic components: East, North, and Vertical.
    - The third dimension (1) is for broadcasting purposes.
    - The fourth dimension corresponds to the number of unique seismic stations.

    Args:
        num_unique_stations (int): The total number of unique stations in the Stream.
        st (obspy.Stream): The input Stream, where traces are ordered by station and in groups of three (East, North, Vertical).

    Returns:
        np.ndarray: A 4D tensor of shape `(400, 3, 1, num_unique_stations)` containing the seismic data for each station.

    Notes:
        - Each station is expected to have exactly three traces in the Stream: East, North, and Vertical components, in that order.
        - The tensor is filled such that for each station, the data from its traces are assigned to the corresponding component in the tensor.
        - The resulting tensor is cast to `np.float32` for efficiency in deep learning applications.
    """
    shape = (400, 3, 1, num_unique_stations)

    # Create the empty array
    X = np.empty(shape)

    x = 0
    for i in range(0, len(st), 3):
        # Extract the components for the current station
        east_trace = st[i]
        north_trace = st[i+1]
        vertical_trace = st[i+2]
    
        # Fill the array X with the respective component data
        X[:, 0, 0, x] = east_trace.data
        X[:, 1, 0, x] = north_trace.data
        X[:, 2, 0, x] = vertical_trace.data

        x+=1

    X = X.astype(np.float32)
    return X

![My Image](../../../Desktop/Tensor.png)

The image depicts the structure of an acceleration waveform tensor used as input for the GRAPES model, which contains seismic data from multiple stations. Each station provides 400 samples representing 4 seconds of data across three waveform channels: East (E), North (N), and Vertical (Z). This data is organized into tensors, allowing the model to process seismic information from multiple stations simultaneously. By capturing waveform features over 4-second windows, the GRAPES model can analyze seismic activity in real-time, utilizing this structured input to predict ground shaking intensity across the network of stations.

In [19]:
def pga_calculator(st):
    """
    Calculate Peak Ground Acceleration (PGA) for each station in the seismic data.

    This function computes the logarithm of the maximum amplitude (PGA) for each station using the three seismic components (East, North, and Vertical). 
    The PGA is calculated by combining the squared amplitude of each component and taking the square root of the sum, followed by calculating the log base 10 of the maximum value.

    Args:
        st (obspy.Stream): The input Stream, where traces are ordered by station and in groups of three (East, North, Vertical).

    Returns:
        list: A list of PGA values (log10 of the maximum amplitude) for each station.

    Notes:
        - The input Stream is expected to have traces grouped by station, with three traces for each station: East, North, and Vertical components.
        - The function computes the combined amplitude for each station by summing the squares of the individual components and then taking the square root of the sum.
        - The logarithm (base 10) of the maximum amplitude is returned for each station.
    """
    pga = []
    for i in range(0, len(st), 3):
        # Extract the components for the current station
        east_trace = st[i]
        north_trace = st[i+1]
        vertical_trace = st[i+2]
    
        # Fill the array X with the respective component data
        amp = east_trace.data ** 2
        amp += north_trace.data ** 2
        amp += vertical_trace.data ** 2
        np.sqrt(amp, out=amp)
        pga.append(np.log10(np.max(amp)))

    return pga

### Running The GRAPES Model

This code chunk processes seismic event data using the GRAPES model and integrates the functions mentioned before for this purpose. It iterates through each event, defining time windows and filtering the seismic network inventory to include only relevant stations. Stream data is segmented into 4-second windows and adjusted to ensure completeness, using functions such as window, sample_adjust_stream, and sorter. Distance calculations generate distance and adjacency matrices, which are then used to prepare tensors and Peak Ground Acceleration (PGA) values. These inputs are fed into the GRAPES model, leveraging the functions tensor_maker, pga_calculator, and adjacency_matrix. The model produces predictions, which, along with actual values, are organized into a dictionary and collected for further analysis.

![My Image](../../../Desktop/Grapes.png)

In the GRAPES Earthquake Early Warning system, both Convolutional Neural Networks (CNNs) and Dense Neural Networks (DNNs) play a crucial role in processing seismic data. The CNN first processes seismic waveform data by analyzing short 4-second windows of ground motion using 1D convolutions. These convolutions identify key patterns, such as the amplitude and phase of seismic waves, that are critical for understanding how the earthquake is progressing. The CNN transforms this raw waveform data into a set of high-level feature vectors, capturing essential information from the waveforms. These feature vectors are then passed to a Dense Neural Network (DNN), which further refines and combines these features. The DNN processes the extracted features to make more accurate predictions about the intensity and spread of the earthquake’s shaking. By stacking layers of fully connected neurons, the DNN helps the model understand complex relationships in the data, improving the accuracy of the final predictions about ground shaking and warning times at different seismic stations.

In [ ]:
#Running GRAPES Model
collection = []
i=0
model = jl.load_GRAPES_model()
for event_key, st in event_streams.items():
    start_of_window, end_of_window = window_arrays(dates[i])
   
    inv = inventory.copy()
   
    # Create a set of station identifiers (network.code) that are present in the stream
    stream_stations = set((trace.stats.network, trace.stats.station) for trace in st)
   
    # Filter the inventory to keep only stations that are in the stream
    for network in inv:
        stations_to_remove = []
        for station in network:
            # Use both network and station code to identify the station
            station_id = (network.code, station.code)
            if station_id not in stream_stations:
                stations_to_remove.append(station)
        for station in stations_to_remove:
            network.stations.remove(station)


    stations_to_remove = []

    prev_station_code = None
   
    for network in inv:
        for station in network.stations:
            if prev_station_code is not None and prev_station_code == station.code:
                #Store the previous station in the list for removal
                stations_to_remove.append((network, prev_station))
            #Set prev_station to the current station
            prev_station = station
            prev_station_code = station.code
   
    #Now remove the stations after the loop
    for network, station in stations_to_remove:
        network.stations.remove(station)

    data_dict = {}
   
    for j in range(len(start_of_window)):
        temp_st = window(start_of_window[j], end_of_window[j], st)
        temp_st = sample_adjust_stream(400, temp_st)

        if len(temp_st) != 0:
            d = temp_st._groupby("{network}.{station}")
            for key, value in d.items():
                if len(value) != 3:
                    for tr in value:
                        temp_st.remove(tr)
       
            num_stations = num_unique_stations(temp_st)
       
            dist_from_epicenter_single, dist_from_epicenter_full, station_location = distance_and_location(temp_st, inv, 38.21550, -122.31167)
           
            temp_st, dist_from_epicenter_single_array, dist_from_epicenter_full_array, station_location_array = sorter(dist_from_epicenter_single, dist_from_epicenter_full, station_location, temp_st)
       
            dist_matrix = distance_matrix(station_location_array)
       
            adj_matrix = adjacency_matrix(dist_matrix, station_location_array)
           
            X = tensor_maker(num_stations, temp_st)
       
            pga = pga_calculator(temp_st)
       
            pred = model(jl.GNNGraph(adj_matrix, ndata = X, gdata= pga))
            pred_values = np.array(pred.ndata.x).flatten()
            actual_values = np.array(pred.gdata.u).flatten()
       
            # Initialize a list to hold all data temporarily
            all_station_data = []
           
            # Main loop
            for k in range(len(station_location_array)):
                station_code = temp_st[3 * k].stats.station
                station_data = {
                    "time_window": j,
                    "location": station_location_array[k],
                    "prediction": pred_values[k],
                    "actual": actual_values[k]
                }
                # Collect the station code and data in a list
                all_station_data.append((station_code, station_data))
           
            # Update data_dict after the loop
            for station_code, station_data in all_station_data:
                if station_code not in data_dict:
                    data_dict[station_code] = []
                data_dict[station_code].append(station_data)

    collection.append(data_dict)
    i+=1

### Results

![My Image](../../../Desktop/PGA.png)

These two maps compare the predicted and actual Peak Ground Acceleration (PGA) values at seismic stations across the region for the Ridgecrest EQ. In the left map, the predicted PGA values (maximum during the event) are shown, while the right map displays the actual recorded PGA. Both maps use a color gradient, with lighter colors indicating higher PGA values (on a logarithmic scale). The red star marks the earthquake’s epicenter. We can see that there is overprediction of PGA from the model.

![My Image](../../../Desktop/1.png)

A scatter plot comparing the observed Modified Mercalli Intensity (MMI) to the predicted MMI values at each station for the Ridgecrest EQ. The colored dots indicate the status of each station’s prediction (True Positive, True Negative, False Positive, False Negative), with red dashed lines marking the MMI threshold of 4. A warning alert is sent out when the model first detects an MMI of at least 4. We can see that the model either unnecessarily sent out an alert or correctly sent out an alert. The model did not have an instance where it did not send out an alert when it should have. This is important as failing to send an alert is the worst possible outcome. 

![My Image](../../../Desktop/3.png)

A scatter plot showing the relationship between warning time (in seconds) and the distance from the earthquake's origin (in kilometers) for the Ridgecrest EQ. The plot demonstrates how warning times increase with distance from the epicenter. This is the desired result.

![My Image](../../../Desktop/Warning.png)

This graph compares the predicted and observed MMI values over time for seismic station CRR during the Ridgecrest EQ. The blue line represents the observed shaking, while the red line shows the predicted peak shaking intensity. The green dashed line marks the MMI = 4 threshold, which indicates when the shaking becomes strong enough to issue warnings. The system predicted the shaking would reach MMI 4 about 40 seconds before it was actually observed at the station, providing a significant warning time.

### Conclusions

The GRAPES algorithm demonstrated an overprediction of Peak Ground Acceleration (PGA) during the Ridgecrest Earthquake and other events in the test suite. This overprediction could be due to the need for retraining on California-specific data, as the region’s geophysical features differs significantly from Japan, where GRAPES was initially tested. However, overprediction in earthquake early warning systems is generally less concerning, as it is better to issue an unnecessary alert than to fail to send one in a critical situation.

Importantly, GRAPES provided good warning times in California. During the Ridgecrest Earthquake, the USGS' ShakeAlert Early Warning System failed to send an alert to Los Angeles County. In contrast, GRAPES successfully issued an alert, providing 30-40 seconds of warning time. This advance notice is crucial in earthquake early warning systems, as every second counts for saving lives, reducing injuries, and allowing people to take protective actions. The ability to deliver timely warnings is one of the most critical aspects of an effective early warning system.